# Tarefa 02

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os monitores
- Divirta-se :)

#### 1. Carregue os dados da tarefa anterior

In [29]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Definindo os ativos do Yahoo Finance no período de 2 anos até a data atual
ativos = ['^BVSP', 'PETR4.SA', 'VALE3.SA']
dt_ini, dt_fim = '2023-01-01', '2025-07-01'

# Baixando os dados
df_bvsp = yf.download(ativos, start=dt_ini, end=dt_fim)

print("Verificando valores nulos:\n")
print(df_bvsp.isnull().sum())

/tmp/ipython-input-2357646279.py:11: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  3 of 3 completed

Verificando valores nulos:

Price   Ticker  
Close   PETR4.SA    0
        VALE3.SA    0
        ^BVSP       0
High    PETR4.SA    0
        VALE3.SA    0
        ^BVSP       0
Low     PETR4.SA    0
        VALE3.SA    0
        ^BVSP       0
Open    PETR4.SA    0
        VALE3.SA    0
        ^BVSP       0
Volume  PETR4.SA    0
        VALE3.SA    0
        ^BVSP       0
dtype: int64


#### 2. Média móvel

1. Faça uma média móvel  em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu
2. Utilizando a fução ```rolling()```, calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
2. Monte um gráfico de linhas para as médias móveis dos ativos da sua base
4. Monte um gráfico de linhas para os desvios padrão em janela móvel
5. Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.

In [30]:
# Faça uma média móvel em 90 dias dos valores de fechamento ajustados dos ativos que você escolheu

# Separando a coluna de fechamento
fechamento = df_bvsp['Close']
fechamento

Ticker,PETR4.SA,VALE3.SA,^BVSP
Date,,,
2023-01-02,13.380753,70.104385,106376.0
2023-01-03,13.042145,69.978912,104166.0
2023-01-04,13.456643,70.104385,105334.0
2023-01-05,13.941199,71.280647,107518.0
2023-01-06,13.859466,72.409843,108836.0
...,...,...,...
2025-06-24,30.674120,48.826660,137165.0
2025-06-25,30.517668,48.768692,135767.0
2025-06-26,30.762121,50.237164,137114.0


In [31]:
# Utilizando a fução rolling(), calcule o desvio pardão desses ativos ao longo do tempo, utilizando a mesma janela móvel do item acima
# Adicionado a média móvel de 90 dias
media_movel_90d = fechamento.rolling(window=90, min_periods=1).mean()
media_movel_90d

Ticker,PETR4.SA,VALE3.SA,^BVSP
Date,,,
2023-01-02,13.380753,70.104385,106376.000000
2023-01-03,13.211449,70.041649,105271.000000
2023-01-04,13.293180,70.062561,105292.000000
2023-01-05,13.455185,70.367083,105848.500000
2023-01-06,13.536041,70.775635,106446.000000
...,...,...,...
2025-06-24,31.444974,52.233153,132250.488889
2025-06-25,31.404994,52.205157,132353.211111
2025-06-26,31.373391,52.197300,132494.700000


In [32]:
# Desvio padrão móvel de 90 dias
desvio_padrao_90d = fechamento.rolling(window=90, min_periods=1).std()
desvio_padrao_90d

Ticker,PETR4.SA,VALE3.SA,^BVSP
Date,,,
2023-01-02,NaN,NaN,NaN
2023-01-03,0.239432,0.088723,1562.705986
2023-01-04,0.220690,0.072442,1105.598480
2023-01-05,0.370744,0.611909,1433.062339
2023-01-06,0.368480,1.056124,1823.535577
...,...,...,...
2025-06-24,2.288610,1.780905,5179.472179
2025-06-25,2.272801,1.815395,5156.208814
2025-06-26,2.261595,1.822430,5109.463402


In [33]:
# Monte um gráfico de linhas para as médias móveis dos ativos da sua base

fig_media = go.Figure()
for ativo in fechamento.columns:
    fig_media.add_trace(go.Scatter(
        x=fechamento.index,
        y=media_movel_90d[ativo],
        mode='lines',
        name=f'{ativo} - MM 90d'
    ))

fig_media.update_layout(
    title='Média Móvel de 90 Dias dos Ativos',
    xaxis_title='Data',
    yaxis_title='Preço',
    template='plotly_dark'
)
fig_media.show()



In [34]:
# Desvio padrão móvel de 90 dias

volatilidade_pct = fechamento.pct_change().rolling(window=90).std()

fig = go.Figure()
for ativo in fechamento.columns:
    fig.add_trace(go.Scatter(
        x=volatilidade_pct.index,
        y=volatilidade_pct[ativo],
        mode='lines',
        name=f'{ativo} - Volatilidade (90d)'
    ))

fig.update_layout(
    title='Volatilidade (Desvio Padrão das Variações Percentuais - 90 Dias)',
    xaxis_title='Data',
    yaxis_title='Volatilidade (%)',
    template='plotly_dark'
)
fig.show()



**Compare os dois gráficos que você fez com o da tarefa anterior. Veja se eles apontam conclusões semelhantes quanto aos períodos de maior volatilidade (variação) dos ativos.**

>Em meados de 2023, o segundo gráfico indica um pico de volatilidade da PETR4 e VALE3, e no primeiro gráfico podemos observar variações maiores nos preços suavizados para esses ativos, especialmente VALE3.

> A média móvel mostra tendências de preço e suavização, enquanto o gráfico de volatilidade indica os períodos e intensidade das variações. Ambos juntos ajudam a validar os períodos de maior volatilidade do mercado e confirmam as conclusões semelhantes quanto aos períodos de maior variação dos ativos.

#### 3. Gráfico de *candlestick*
1. Selecione o período dos últimos 60 dias (corridos) para um dos ativos da sua base
2. Monte um gráfico de *candlestick* para esses dados.

In [18]:
hoje = datetime.today().date()
inicio = hoje - timedelta(days=90)
df_candle = yf.download('PETR4.SA', start=inicio, end=hoje)


df_candle_simple = pd.DataFrame({
    'Open': df_candle['Open']['PETR4.SA'],
    'High': df_candle['High']['PETR4.SA'],
    'Low': df_candle['Low']['PETR4.SA'],
    'Close': df_candle['Close']['PETR4.SA']
}, index=df_candle.index)

fig = go.Figure(data=[go.Candlestick(
    x=df_candle_simple.index,
    open=df_candle_simple['Open'],
    high=df_candle_simple['High'],
    low=df_candle_simple['Low'],
    close=df_candle_simple['Close'],
    name='PETR4.SA'
)])

fig.update_layout(
    title='Gráfico Candlestick - PETR4.SA (Últimos 60 dias)',
    xaxis_title='Data',
    yaxis_title='Preço',
    xaxis_rangeslider_visible=False,
    template='plotly_dark'
)

fig.show()


/tmp/ipython-input-3359695912.py:3: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
